In [1]:
from openai import OpenAI
import pandas as pd
from collections import defaultdict
import tiktoken
import numpy as np
import json

In [2]:
client = OpenAI()

In [40]:
dataset = pd.read_csv('datasets/faqsUcfDataset_2.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       102 non-null    object
 1   A       102 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


In [41]:
dataset.head()

,Q,A
0,Are dual enrollment college transcripts requir...,If you completed dual enrollment coursework wh...
1,Do I need to have my international academic cr...,International high school credentials are eval...
2,"Does UCF Admissions require recommendations, e...",Letters of recommendation are not required and...
3,Does UCF guarantee admission based upon class ...,UCF no longer guarantees admission for Florida...
4,Does UCF have specific essay or personal state...,"Although optional, an essay can assist the Adm..."


In [42]:
dataset["json"] = '{"prompt": "' + dataset["Q"] + '", "completion": "' + dataset["A"] + '"}'

In [43]:
dataset = dataset.drop(["Q", "A"], axis=1)
dataset["json"].apply(json.loads)
jsonlist = dataset["json"].tolist()
jsonlist = [json.loads(x) for x in jsonlist]

In [44]:
with open('faq.jsonl', 'w') as outfile:
    for entry in jsonlist:
        json.dump(entry, outfile)
        outfile.write('\n')

In [46]:
client.files.create(
  file=open("faq.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-GD6MayeJUI27RkPTybru9pNM', bytes=56146, created_at=1710213413, filename='faq.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [47]:
client.fine_tuning.jobs.create(
  training_file="file-GD6MayeJUI27RkPTybru9pNM", 
  model="babbage-002"
)

BadRequestError: Error code: 400 - {'error': {'message': 'Fine-tuning jobs cannot be created on an Explore plan. You can upgrade to a paid plan on your billing page: https://platform.openai.com/account/billing/overview', 'type': 'invalid_request_error', 'param': None, 'code': 'exceeded_quota'}}

In [4]:
messages = [{"role": "system", "content": "You are an intelligent university assistant chatbot"}]

In [ ]:
while True:
    message = input("User > ")
    if message:
        messages.append({"role": "user", "content": message})
        response = client.chat.completions.create(model="gpt-3.5-turbo", messages=messages, temperature=0.25, top_p=1)
        reply = response.choices[0].message.content
        print("Assistant > ", reply)
        messages.append({"role": "assistant", "content": reply})